In [33]:
# # get json file with masks links to download and description  
# !gdrive download --path "/home/jupyter/data/" 1NyKLf3khaszjCUWKUnEnctg7v4YnGqae

In [22]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 26 18:01:51 2019

@author: GANESHA
"""
import json
import requests
from PIL import Image 
import numpy as np
import os
from io import BytesIO
import matplotlib.pyplot as plt
from itertools import groupby

In [23]:
# load the json file with labelbox files descriptors
with open('/home/jupyter/data/labeled_image_data_mask-2019-07-19T14_32_47.510Z.json') as f:
    data = json.load(f)

## Json description file analysis

In [24]:
## What are the possible Mask key combinations:
masks_dicts = []
for el in data:
    if 'Masks' in el:
        keys_list = []
        for keys in el['Masks']:
            keys_list.append(keys)
        masks_dicts.append(keys_list)

In [25]:
# out of above, what are the unique combinations:
output = []
for x in masks_dicts:
    if x not in output:
        output.append(x)
print('Unique sets of masks:')        
print(output)

Unique sets of masks:
[['lc_cat1', 'Non-agricultural (O)', 'Cloud'], ['Non-agricultural (O)'], ['lc_cat1', 'Cloud'], ['lc_cat1', 'Non-agricultural (O)'], ['lc_cat1'], ['Non-agricultural (O)', 'Cloud'], ['lc_cat1', 'Harvested land'], ['lc_cat1', 'Non-agricultural (O)', 'Harvested land'], ['Harvested land'], ['Non-agricultural (O)', 'Harvested land'], ['lc_cat1', 'Non-agricultural (O)', 'Cloud', 'Harvested land'], ['Cloud'], ['Non-agricultural (O)', 'Cloud', 'Harvested land'], ['lc_cat1', 'Cloud', 'Harvested land']]


In [26]:
unique_elements = []
for inner_l in masks_dicts:
    for item in inner_l:
        unique_elements.append(item)
unique_elements = set(unique_elements)
print('Unique Masks keys on the list: {}'.format(list(unique_elements)))

Unique Masks keys on the list: ['Harvested land', 'Cloud', 'lc_cat1', 'Non-agricultural (O)']


## Download masks and original files from Labelbox

Script by Sai with my minor changes. Credits to Sai.

In [32]:
%%time

output_dir = ('/home/jupyter/data/WFP_Nepal_Scale3.5_RGB_and_masks_PNG/')
 
# selecting the unique IDs to have the unique pictures - gives 144 images at the end of this block
ids = []
ext_ids = []
idnots = []
 
for i in range(len(data)):
    if data[i]['External ID'] in ext_ids:
        idnots.append(data[i]['ID'])
    else :
        ext_ids.append(data[i]['External ID'])    
        ids.append(data[i]['ID'])    
        
files_generated = []
keys_used = []

# Just adapted from previous code available at private repo
# This section retrieves images and masks from Labelbox using JSON file
# At first I disregarded the "Unknown" category - just considered the "Trees" categories
for idx, d in enumerate(data):
    if d['ID'] in ids:
        if(d['Label']!='Skip'):
            if('Masks' in d):

                # print("Image: {}, ID: {}, filename: {}".format(idx, d["ID"], d["External ID"]))
                response_og = requests.get(d["Labeled Data"])
                img_og = Image.open(BytesIO(response_og.content))  

                # img_og.save(os.path.join(output_dir, '{}_original.png'.format(idx)), format='png')

                ## added LM to give same as original file names
                # file_name_base = '{}_{}'.format(str(idx), d["External ID"].rstrip('.png'))
                file_name_base = d["External ID"].rstrip('.png')
                img_og.save(os.path.join(output_dir, '{}_original.png'.format(file_name_base)), format='png')              

                # to check generated files.
                files_generated.append(file_name_base)
                
                img_shape = np.array(img_og).shape 
                img_mask = np.zeros((img_shape[0], img_shape[1]))
                img_p1, img_p2, img_p3, img_p4 = np.zeros(img_mask.shape), np.zeros(img_mask.shape), np.zeros(img_mask.shape), np.zeros(img_mask.shape)

                threshold = 200
                
                # Iterating over keys of a Mask key: 
                for key in d["Masks"]:
                    response = requests.get(d["Masks"][key])
                    img = Image.open(BytesIO(response.content)) 
                    if key in ['lc_cat1']:
                        keys_used.append('lc_cat1')
                        img_p1 = np.where(np.array(img)[:, :, 3] > threshold, 255, 0)
                    elif key in ['Non-agricultural (O)']:
                        keys_used.append('Non-agricultural (O)')
                        img_p2 = np.where(np.array(img)[:, :, 3] > threshold, 255, 0)
                    elif key in ['Cloud']:
                        keys_used.append('Cloud')
                        img_p3 = np.where(np.array(img)[:, :, 3] > threshold, 255, 0)
                    elif key in ['Harvested land']:
                        img_p4 = np.where(np.array(img)[:, :, 3] > threshold, 255, 0)
                        keys_used.append('Harvested land')

                img_p = img_p1 + img_p2 + img_p3 + img_p4 - np.where(img_p1 + img_p2 + img_p3 + img_p4 > 255, 255, 0)
                img_final = img_p
                img = Image.fromarray(img_final.astype(np.uint8)).convert(mode='L', dither=Image.NONE)
                img.save(os.path.join(output_dir, '{}_mask.png'.format(file_name_base)), format='png')

## SUMMARY: 
# how many items per category:
files_generated_prefixes = [i.split('_')[0] for i in files_generated]
results = {value: len(list(freq)) for value, freq in groupby(sorted(files_generated_prefixes))}
print('Number of IDs in json file: {}\nValid files with masks {}\nNumber of file types generated per category: {}'.format(len(data), len(files_generated), results))

Number of IDs in json file: 513
Valid files with masks 398
Number of file types generated per category: {'other': 69, 'rice': 301, 'wheat': 28}
CPU times: user 52.2 s, sys: 1.36 s, total: 53.6 s
Wall time: 5min 44s


In [14]:
# display an exemplary ndarray:

from matplotlib import pyplot as plt

# load an image
img = Image.open(BytesIO(response.content))
#x = np.zeros(np.array(img).shape)+ 
x = np.array(img)[:, :, 3]

# show image
plt.imshow(x, interpolation='nearest')
plt.show()